In [6]:
from pyspark.sql import SparkSession

# Initialize a SparkSession
spark = SparkSession.builder \
    .appName("MIND Dataset Processing") \
    .getOrCreate()

24/12/09 15:35:43 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [7]:
# Define the path to the news.tsv file
news_path = "data/mind/MINDsmall_train/news.tsv"

# Define column names for the news.tsv file
news_columns = ["NewsID", "Category", "Subcategory", "Title", "Abstract", "URL", "TitleEntities", "AbstractEntities"]

# Load the news.tsv file into a Spark DataFrame
news_df = spark.read.csv(
    news_path,
    sep="\t",
    schema="NewsID STRING, Category STRING, Subcategory STRING, Title STRING, Abstract STRING, URL STRING, TitleEntities STRING, AbstractEntities STRING",
    header=False
)

# Assign column names
news_df.show(n=2, truncate=True) 

AnalysisException: Path does not exist: file:/app/als_container/data/mind/MINDsmall_train/news.tsv

In [ ]:
news_df.columns

In [ ]:
#### PRE PROCESSING ###

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, regexp_replace, length, size, udf
from pyspark.sql.types import StringType, ArrayType, IntegerType
from pyspark.ml.feature import Tokenizer, StopWordsRemover

import json

In [ ]:
# Initialize Spark session
spark = SparkSession.builder.appName("PreprocessingPipeline").getOrCreate()

# Load your data (modify the path as necessary)
news_df = spark.read.csv("data/mind/MINDsmall_train/news.tsv", sep="\t", header=False, inferSchema=True)

# Assign column names
news_df = news_df.toDF("NewsID", "Category", "Subcategory", "Title", "Abstract", "URL", "TitleEntities", "AbstractEntities")

# Display initial rows
news_df.show(5, truncate=True)

In [ ]:
### MISSING VALUES ###

# Drop rows where Title or Abstract are missing
news_df = news_df.na.drop(subset=["Title", "Abstract"])

# Verify the results
print(f"Rows after dropping missing values: {news_df.count()}")


In [ ]:
### TEXT CLEANING ###

# Define a function to clean text (remove special characters and convert to lowercase)
def clean_text(text):
    if text:
        return text.lower().replace("\n", " ").replace("\t", " ")
    return None

# Register the UDF
clean_text_udf = udf(lambda x: clean_text(x), StringType())

# Apply text cleaning to Title and Abstract
news_df = news_df.withColumn("CleanTitle", clean_text_udf(col("Title")))
news_df = news_df.withColumn("CleanAbstract", clean_text_udf(col("Abstract")))

# Display cleaned text
news_df.select("CleanTitle", "CleanAbstract").show(5, truncate=True)

In [ ]:
### TOKENIZATION ###

# Tokenize CleanTitle and CleanAbstract
tokenizer_title = Tokenizer(inputCol="CleanTitle", outputCol="TitleTokens")
tokenizer_abstract = Tokenizer(inputCol="CleanAbstract", outputCol="AbstractTokens")

news_df = tokenizer_title.transform(news_df)
news_df = tokenizer_abstract.transform(news_df)

# Display tokenized data
news_df.select("TitleTokens", "AbstractTokens").show(5, truncate=True)


In [ ]:
### STOPWORDS REMOVAL ###

# Remove stopwords from TitleTokens and AbstractTokens
stopword_remover_title = StopWordsRemover(inputCol="TitleTokens", outputCol="FilteredTitleTokens")
stopword_remover_abstract = StopWordsRemover(inputCol="AbstractTokens", outputCol="FilteredAbstractTokens")

news_df = stopword_remover_title.transform(news_df)
news_df = stopword_remover_abstract.transform(news_df)

# Display filtered tokens
news_df.select("FilteredTitleTokens", "FilteredAbstractTokens").show(5, truncate=True)


In [ ]:
# Define a UDF to clean each token in the array
def clean_tokens(tokens):
    if tokens:
        return [token.replace(",", "") for token in tokens]  # Remove commas
    return tokens

# Register the UDF
clean_tokens_udf = udf(clean_tokens, ArrayType(StringType()))

# Apply the UDF to FilteredTitleTokens
news_df = news_df.withColumn("FilteredTitleTokens", clean_tokens_udf(col("FilteredTitleTokens")))
news_df = news_df.withColumn("FilteredAbstractTokens", clean_tokens_udf(col("FilteredAbstractTokens")))

In [ ]:
news_df.select("Title", "FilteredTitleTokens").show(5, truncate=True)

In [ ]:
news_df.select("Abstract", "FilteredAbstractTokens").show(2, truncate=True)

In [ ]:
news_df.columns

In [ ]:
news_df.show(1, truncate=True)

In [ ]:
### EMBEDDINGS ###

In [ ]:
from pyspark.sql.functions import concat_ws
from pyspark.ml.feature import CountVectorizer
from pyspark.sql.functions import split
from pyspark.ml.feature import IDF

In [ ]:
# combine tokens 
news_df = news_df.withColumn("CombinedTokens", concat_ws(" ", "FilteredTitleTokens", "FilteredAbstractTokens"))

# Show the updated DataFrame
news_df.select("CombinedTokens").show(5, truncate=True)

In [ ]:
# Convert CombinedTokens (string) into an array of tokens
news_df = news_df.withColumn("CombinedWords", split(news_df["CombinedTokens"], " "))


In [ ]:
# Compute term frequency (TF) using CountVectorizer
cv = CountVectorizer(inputCol="CombinedWords", outputCol="RawFeatures", vocabSize=10000, minDF=2)
cv_model = cv.fit(news_df)
news_df = cv_model.transform(news_df)

# Show the resulting term frequency vector
news_df.select("NewsID", "RawFeatures").show(5, truncate=True)

In [ ]:
idf = IDF(inputCol="RawFeatures", outputCol="TFIDFeatures")
idf_model = idf.fit(news_df)
news_df = idf_model.transform(news_df)

# Show the resulting DataFrame with TF-IDF features
news_df.select("NewsID", "RawFeatures").show(5, truncate=True)

In [ ]:
spark.stop()

In [22]:
### BERT EMBEDDING ###

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("BertEmbeddingExample") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:4.2.6") \
    .getOrCreate()


24/12/09 15:35:10 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
news_df = news_df.withColumn("text", concat_ws(" ", col("CombinedWords")))

NameError: name 'news_df' is not defined

In [21]:
news_df.columns

['NewsID',
 'Category',
 'Subcategory',
 'Title',
 'Abstract',
 'URL',
 'TitleEntities',
 'AbstractEntities',
 'CleanTitle',
 'CleanAbstract',
 'TitleTokens',
 'AbstractTokens',
 'FilteredTitleTokens',
 'FilteredAbstractTokens',
 'CombinedTokens',
 'CombinedWords',
 'RawFeatures',
 'TFIDFeatures']

In [22]:
## NB: example (10000, [310, 978, 1119], [0.5, 1.2, 0.8])
#This means:
#The vocabulary size is 10,000.
#Words at indices 310, 978, and 1119 in the vocabulary are present in the document.
#Their respective TF-IDF scores are 0.5, 1.2, and 0.8.


In [34]:
# Save DataFrame to CSV
news_df.write.parquet("news", mode="overwrite")

[Stage 26:>                                                         (0 + 8) / 8]

24/12/09 14:22:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [36]:
# Path to the saved Parquet file
parquet_path = "news"

# Read the Parquet file into a DataFrame
tfidf_df = spark.read.parquet(parquet_path)

# Show the schema of the loaded DataFrame
tfidf_df.show(1, truncate = True)


+------+--------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------------+--------------------+--------------------+--------------------+--------------------+
|NewsID|Category|        Subcategory|               Title|            Abstract|                 URL|       TitleEntities|    AbstractEntities|          CleanTitle|       CleanAbstract|         TitleTokens|      AbstractTokens| FilteredTitleTokens|FilteredAbstractTokens|      CombinedTokens|       CombinedWords|         RawFeatures|        TFIDFeatures|
+------+--------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------------+--------------------